## Import

In [13]:
# !pip3 install albumentations ;!pip3 install opencv-python ; !pip3 install tqdm

In [14]:
import wandb
wandb.login()
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Dacon-papering-competition",
    
    # track hyperparameters and run metadata
    config={
    'LEARNING_RATE':3e-4,
    'IMG_SIZE':224,
    "architecture": "Efficientnet-b0",
    'EPOCHS':30,
    'SEED':42,
    'BATCH_SIZE':64,
    "dataset": "Dacon-papering-competition",
    }# hyperparameter
)

# hyperparameter
CFG = wandb.config
CFG

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


{'LEARNING_RATE': 0.0003, 'IMG_SIZE': 224, 'architecture': 'Efficientnet-b0', 'EPOCHS': 30, 'SEED': 42, 'BATCH_SIZE': 64, 'dataset': 'Dacon-papering-competition'}

In [15]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
from torch.autograd import Variable


from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

# torch.multiprocessing import
from torch import multiprocessing

import warnings
warnings.filterwarnings(action='ignore') 

In [16]:
# 경로지정
import os
os.chdir('../DATA')
os.getcwd()

'c:\\Users\\DoSungjin\\Documents\\GitHub\\Dacon_papering_classification\\DATA'

In [17]:
# seeds
random_seed = 42
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)


In [18]:
# Set the device to CPU or GPU depending on availability
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.is_available()

True

## Hyperparameter Setting

## Fixed RandomSeed

In [19]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [20]:
all_img_list = glob.glob('/ori_train/*/*')

In [21]:
os.getcwd()

'c:\\Users\\DoSungjin\\Documents\\GitHub\\Dacon_papering_classification\\DATA'

In [22]:
import os

# 데이터셋 디렉토리 경로
dataset_dir = "train"

# 모든 이미지 파일 경로 리스트
all_img_list = []
folder_list = []
train_file_list = os.listdir(dataset_dir)
for item in train_file_list:
    if not item.startswith('.'):
        item_path = os.path.join(dataset_dir, item)
        for file in os.listdir(item_path):
            if not file.startswith('.'):
                all_img_list.append(os.path.join(item_path, file))
                folder_list.append(item)

In [23]:
all_img_list[:5]

['train\\0\\0.png',
 'train\\0\\1.png',
 'train\\0\\10.png',
 'train\\0\\11.png',
 'train\\0\\2.png']

In [24]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x: int(str(x).split('\\')[-2]))
df['label'] = df['label'].astype(torch.int64)

df.head()

TypeError: Cannot interpret 'torch.int64' as a data type

In [ ]:
type(df.label[0])

numpy.int64

In [ ]:
# train val split
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, stratify=df['label'], random_state=CFG['SEED'])


In [ ]:
train

,img_path,label
2962,train\3\163.png,3
1400,train\18\1051.png,18
118,train\1\194.png,1
1004,train\11\52.png,11
1427,train\18\1076.png,18
...,...,...
1595,train\18\1227.png,18
491,train\10\253.png,10
1951,train\18\284.png,18
1639,train\18\1267.png,18


## Label-Encoding

In [ ]:
# le = preprocessing.LabelEncoder()
# train['label'] = le.fit_transform(train['label'])
# val['label'] = le.transform(val['label'])
train

,img_path,label
2962,train\3\163.png,3
1400,train\18\1051.png,18
118,train\1\194.png,1
1004,train\11\52.png,11
1427,train\18\1076.png,18
...,...,...
1595,train\18\1227.png,18
491,train\10\253.png,10
1951,train\18\284.png,18
1639,train\18\1267.png,18


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2765 entries, 2962 to 849
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   img_path  2765 non-null   object
 1   label     2765 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 64.8+ KB


In [ ]:
# label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
# print("Label mapping:")
# print(label_mapping)
# # train_folder = 'train'
# # # 작업 디렉토리 변경
# # os.chdir(train_folder)

# # 폴더명 변경
# for old_name, new_label in label_mapping.items():
#     os.rename(old_name, str(new_label))

In [ ]:
import json
#불러올 파일 경로
input_file = 'label_mapping.json'

# JSON 파일 불러오기
with open(input_file, 'r') as f:
    label_mapping = json.load(f)

print("Label mapping loaded from", input_file)
print(label_mapping)

Label mapping loaded from label_mapping.json
{'가구수정': 0, '걸레받이수정': 1, '곰팡이': 2, '꼬임': 3, '녹오염': 4, '들뜸': 5, '면불량': 6, '몰딩수정': 7, '반점': 8, '석고수정': 9, '오염': 10, '오타공': 11, '울음': 12, '이음부불량': 13, '창틀,문틀수정': 14, '터짐': 15, '틈새과다': 16, '피스': 17, '훼손': 18}


In [ ]:
os.getcwd()

'c:\\Users\\DoSungjin\\Documents\\GitHub\\Dacon_papering_classification\\DATA'

## CustomDataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
train

,img_path,label
2962,train\3\163.png,3
1400,train\18\1051.png,18
118,train\1\194.png,1
1004,train\11\52.png,11
1427,train\18\1076.png,18
...,...,...
1595,train\18\1227.png,18
491,train\10\253.png,10
1951,train\18\284.png,18
1639,train\18\1267.png,18


In [ ]:
#weighted sampling
a_list = train['label'].value_counts()
class_counts = train['label'].value_counts().to_list()
class_weights_dict = {label: 1/count for label, count in a_list.items()}

labels_lists = train['label'].to_list()
weights = [class_weights_dict[label] for label in labels_lists]
num_samples = sum(class_counts)
sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples), replacement=True)
sampler

In [ ]:
# make dataloader
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], sampler=sampler, shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)

val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha, (float, int)):
            self.alpha = torch.Tensor([alpha] * 19)
        self.alpha[18] = 1 - alpha
        if isinstance(alpha, list):
            self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim() > 2:
            input = input.view(input.size(0), input.size(1), -1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1, 2)  # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1, input.size(2))  # N,H*W,C => N*H*W,C

        if not isinstance(target, torch.Tensor):
            target = torch.tensor(target, dtype=torch.long)

        target = target.view(-1, 1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1, target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type() != input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0, target.data.view(-1))
            logpt = logpt * at

        loss = -1 * (1 - pt) ** self.gamma * logpt
        if self.size_average:
            return loss.mean()
        else:
            return loss.sum()


## Model Define

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(df['label'].unique())):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Train

In [ ]:
# def train(model, optimizer, train_loader, val_loader, scheduler, device):
#     model.to(device)
#     criterion = nn.CrossEntropyLoss().to(device)
    
#     best_score = 0
#     best_model = None
    
#     for epoch in range(1, CFG['EPOCHS']+1):
#         model.train()
#         train_loss = []
#         for imgs, labels in tqdm(iter(train_loader)):
#             imgs = imgs.float().to(device)
            
#             optimizer.zero_grad()
            
#             output = model(imgs)
#             loss = criterion(output, labels)
            
#             loss.backward()
#             optimizer.step()
            
#             train_loss.append(loss.item())
                    
#         _val_loss, _val_score = validation(model, criterion, val_loader, device)
#         _train_loss = np.mean(train_loss)
#         print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
#         if scheduler is not None:
#             scheduler.step(_val_score)
            
#         if best_score < _val_score:
#             best_score = _val_score
#             best_model = model
    
#     return best_model

In [ ]:
## Train

def trains(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
#     criterion = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)
    criterion = FocalLoss(gamma=2, alpha=0.25)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            # labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [ ]:
os.getcwd()

'c:\\Users\\DoSungjin\\Documents\\GitHub\\Dacon_papering_classification\\DATA'

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            # labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha, (float, int)):
            self.alpha = torch.Tensor([alpha] * 19)
        self.alpha[18] = 1 - alpha
        if isinstance(alpha, list):
            self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim() > 2:
            input = input.view(input.size(0), input.size(1), -1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1, 2)  # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1, input.size(2))  # N,H*W,C => N*H*W,C

        if not isinstance(target, torch.Tensor):
            target = torch.tensor(target, dtype=torch.long).view(-1)  # Modify this line

        target = target.view(-1, 1).float()

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1, target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type() != input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0, target.data.view(-1))
            logpt = logpt * at

        loss = -1 * (1 - pt) ** self.gamma * logpt
        if self.size_average:
            return loss.mean()
        else:
            return loss.sum()


def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    val_preds = []
    val_targets = []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = torch.tensor(labels, dtype=torch.int64)
            labels = labels.to(device)

            output = model(imgs)
            loss = criterion(output, labels)

            val_loss.append(loss.item())

            preds = torch.argmax(output, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_targets.extend(labels.cpu().numpy())

    val_loss = np.mean(val_loss)
    val_score = calculate_score(val_targets, val_preds)  # Replace with your own score calculation function

    return val_loss, val_score


def trains(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = FocalLoss(gamma=2, alpha=0.25)

    best_score = 0
    best_model = None

    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = torch.tensor(labels, dtype=torch.int64)
            labels = labels.to(device)
        

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model


In [ ]:
# # #!/bin/bash
# # os.chdir('train')
# # 변경 전 폴더명 리스트
# old_names = ["녹오염", "걸레받이수정", "꼬임", "석고수정", "오타공", "울음", "이음부불량",
#              "몰딩수정", "면불량", "창틀,문틀수정", "피스", "곰팡이", "반점", "들뜸", "오염",
#              "가구수정", "터짐", "훼손", "틈새과다"]

# # 변경 후 폴더명 리스트
# new_names = ["rust", "mop_cleaning", "twist", "gypsum_repair", "typo", "cry", "splice_defect",
#              "molding_repair", "wrinkle", "window_frame_repair", "piece", "mold", "stain", 
#              "uneven", "contamination", "furniture_repair", "crack", "damage", "excessive_gap"]

# ch_names = os.listdir()
# # 현재 폴더 경로
# current_dir = os.getcwd()


# # # 폴더명 변경
# # for old_name, new_name in zip(ch_names, old_names):
# #     old_path = old_name
# #     new_path = new_name
# #     os.rename(old_path, new_path)




## Run!!

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = trains(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/44 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 74.00 MiB (GPU 0; 8.00 GiB total capacity; 7.18 GiB already allocated; 0 bytes free; 7.25 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Inference

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = [key for key, value in label_mapping.items() if value in preds]
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

## Submission

In [ ]:
submit = pd.read_csv('sample_submission.csv')

In [ ]:
submit['label'] = preds

In [ ]:
# result folder 생성
from datetime import datetime, timezone, timedelta

kst = timezone(timedelta(hours=9))
train_serial =  datetime.now(kst).strftime('%Y%m%d_%H%M%S')

Record_path = os.path.join('../result', train_serial)

os.makedirs(Record_path, exist_ok=True)




In [ ]:
submit

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,몰딩수정
4,TEST_004,오염
...,...,...
787,TEST_787,꼬임
788,TEST_788,훼손
789,TEST_789,오염
790,TEST_790,오염


In [ ]:
submit.to_csv(os.path.join(Record_path,'submit.csv'), index=False)